![Neptune + TensorBoard](https://neptune.ai/wp-content/uploads/2023/09/tensorboard_tensorflow.svg)

# Neptune + TensorBoard (Tensorflow)

<a target="_blank" href="https://colab.research.google.com/github/neptune-ai/examples/blob/main/integrations-and-supported-tools/tensorboard/notebooks/Neptune_Tensorflow_Tensorboard.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a><a target="_blank" href="https://github.com/neptune-ai/examples/blob/main/integrations-and-supported-tools/tensorboard/notebooks/Neptune_Tensorflow_Tensorboard.ipynb">
  <img alt="Open in GitHub" src="https://img.shields.io/badge/Open_in_GitHub-blue?logo=github&labelColor=black">
</a><a target="_blank" href="https://app.neptune.ai/o/common/org/tensorboard-integration/runs/details?viewId=standard-view&detailsTab=metadata&shortId=TBOARD-880&type=run"> 
  <img alt="Explore in Neptune" src="https://neptune.ai/wp-content/uploads/2024/01/neptune-badge.svg">
</a><a target="_blank" href="https://docs.neptune.ai/integrations/tensorboard/">
  <img alt="View tutorial in docs" src="https://neptune.ai/wp-content/uploads/2024/01/docs-badge-2.svg">
</a>

Introduction

This guide will show you how to:

* Use the `enable_tensorboard_logging()` function to log metrics and metadata to both TensorBoard and Neptune
* Export previous Tensorboard logs to Neptune

## Before you start

This notebook example lets you try out Neptune anonymously, with zero setup.

If you want to see the example logged to your own workspace instead:

  1. Create a Neptune account. [Register &rarr;](https://neptune.ai/register)
  1. Create a Neptune project that you will use for tracking metadata. For instructions, see [Creating a project](https://docs.neptune.ai/setup/creating_project) in the Neptune docs.

## Install Neptune and dependencies

In [ ]:
%pip install -U neptune[tensorboard]

## Start a run

To create a new run for tracking the metadata, you tell Neptune who you are (`api_token`) and where to send the data (`project`).

You can use the default code cell below to create an anonymous run in a public project. **Note**: Public projects are cleaned regularly, so anonymous runs are only stored temporarily.

### Log to your own project instead

Replace the code below with the following:

```python
import neptune
from getpass import getpass

run = neptune.init_run(
    project="workspace-name/project-name",  # replace with your own (see instructions below)
    api_token=getpass("Enter your Neptune API token: "),
)
```

To find your API token and full project name:

1. [Log in to Neptune](https://app.neptune.ai/).
1. In the bottom-left corner, expand your user menu and select **Get your API token**.
1. The workspace name is displayed in the top-left corner of the app. To copy the project path, in the top-right corner, open the settings menu and select **Properties**.

For more help, see [Setting Neptune credentials](https://docs.neptune.ai/setup/setting_credentials) in the Neptune docs.


In [ ]:
import neptune

run = neptune.init_run(
    api_token=neptune.ANONYMOUS_API_TOKEN,  # replace with your own
    project="common/tensorboard-integration",  # replace with your own
    tags=["notebook", "sync"],  # optional
)

**To open the run in the Neptune web app, click the link that appeared in the cell output.**

We'll use the `run` object we just created to log metadata. You'll see the metadata appear in the app.

### Imports

In [ ]:
import io

import numpy as np
import requests
import tensorflow as tf
from neptune_tensorboard import enable_tensorboard_logging

### Enable Neptune integration with TensorBoard to log to both Neptune and TensorBoard
Calling `enable_tensorboard_logging()` also works with SummaryWriter from PyTorch and tensorboardX and also `Tensorboard` callback for Keras.
__NOTE:__ This will log to both the `tensorboard` directory and the Neptune run.

In [ ]:
enable_tensorboard_logging(run)

### Create default writer

In [ ]:
writer = tf.summary.create_file_writer("logs")
writer.set_as_default(0)

### Data for training

In [ ]:
response = requests.get("https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz")
with open("mnist.npz", "wb") as f:
    f.write(response.content)


with np.load("mnist.npz") as data:
    train_examples = data["x_train"]
    train_labels = data["y_train"]
    test_examples = data["x_test"]
    test_labels = data["y_test"]

### Hyperparameters for training

In [ ]:
params = {
    "batch_size": 1024,
    "shuffle_buffer_size": 100,
    "lr": 0.001,
    "num_epochs": 5,
    "num_visualization_examples": 10,
}

# Log training parameters
for name, value in params.items():
    tf.summary.scalar(name, value)

### Prepare data for training

In [ ]:
# Normalize data for training
def normalize_img(image):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.cast(image, tf.float32) / 255.0


train_examples = normalize_img(train_examples)
test_examples = normalize_img(test_examples)

# Prepare data for training
train_dataset = tf.data.Dataset.from_tensor_slices((train_examples, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_examples, test_labels))

train_dataset = train_dataset.shuffle(params["shuffle_buffer_size"]).batch(params["batch_size"])
test_dataset = test_dataset.batch(params["batch_size"])

### Prepare Model

In [ ]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(10),
    ]
)

# Loss
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Optimizer
optimizer = tf.keras.optimizers.Adam(params["lr"])

# Log model summary
with io.StringIO() as s:
    model.summary(print_fn=lambda x: s.write(x + "\n"))
    model_summary = s.getvalue()

tf.summary.text("model_summary", model_summary)

### Helper functions for training

In [ ]:
# Helper functions for training loop
def loss_and_preds(model, x, y, training):
    # training=training is needed only if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    y_pred = model(x, training=training)

    return loss_object(y_true=y, y_pred=y_pred), y_pred


def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value, _ = loss_and_preds(model, inputs, targets, training=True)
    return loss_value, tape.gradient(loss_value, model.trainable_variables)

### Training loop

In [ ]:
for epoch in range(params["num_epochs"]):
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

    for x, y in train_dataset:
        loss_value, grads = grad(model, x, y)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        epoch_loss_avg.update_state(loss_value)
        epoch_accuracy.update_state(y, model(x, training=True))

    # Log metrics for the epoch
    # Train metrics
    tf.summary.scalar("loss", epoch_loss_avg.result())
    tf.summary.scalar("accuracy", epoch_accuracy.result())

    # Log test metrics
    test_loss, test_preds = loss_and_preds(model, test_examples, test_labels, False)
    test_acc = epoch_accuracy(test_labels, test_preds)
    tf.summary.scalar("test_loss", test_loss)
    tf.summary.scalar("test_accuracy", test_acc)

    # Log test prediction
    for idx in range(params["num_visualization_examples"]):
        np_image = test_examples[idx].numpy().reshape(1, 28, 28, 1)
        pred_label = test_preds[idx].numpy().argmax()
        true_label = test_labels[idx]
        tf.summary.image(f"epoch-{epoch}_pred-{pred_label}_actual-{true_label}", np_image)

    if epoch % 5 == 0 or epoch == (params["num_epochs"] - 1):
        print(
            "Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(
                epoch, epoch_loss_avg.result(), epoch_accuracy.result()
            )
        )

## Stop logging

Once you are done logging, stop tracking the run.

In [ ]:
run.stop()

## Explore the results in Neptune

Follow the run link to explore the logged metadata (metrics, params, predictions) in Neptune.

You can also check out an [example run](https://app.neptune.ai/o/common/org/tensorboard-integration/runs/details?viewId=standard-view&detailsTab=metadata&shortId=TBOARD-880&type=run).

## Exporting previous TensorBoard Logs
Export previous logs in the `logs` directory with the `neptune tensorboard` command via CLI.
__NOTE:__ Passing the `api_token` and `project` arguments can be skipped if the corresponding environment variables are set.

In [ ]:
# !neptune tensorboard --api_token API_TOKEN --project PROJECT_NAME logs